In [68]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools

df=pd.read_csv("dati_milano/milano_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [69]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))


In [70]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe 1"
    else: prezzi_inferiori = "Dataframe 2"
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len
    dict_df['tipo ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_differenze = pd.DataFrame(list_dict_df)   
df_differenze.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo ricerche,prezzi_inferiori
9,03-02-2024---20:31 *** 03-02-2024---20:42,712,8.571629,7.0,0.740125,desktop/desktop,Dataframe 2
20,03-02-2024---20:42 *** 03-02-2024---22:10,703,8.150782,7.0,0.736897,desktop/desktop,Dataframe 1
21,03-02-2024---20:42 *** 03-02-2024---22:41,697,8.711621,7.0,0.784910,desktop/mobile,Dataframe 1
19,03-02-2024---20:42 *** 03-02-2024---21:42,691,8.798842,7.0,0.786121,desktop/mobile,Dataframe 1
17,03-02-2024---20:42 *** 03-02-2024---20:53,689,9.001451,7.0,0.775901,desktop/mobile,Dataframe 2
18,03-02-2024---20:42 *** 03-02-2024---21:30,678,8.467552,7.0,0.739368,desktop/desktop,Dataframe 1
22,03-02-2024---20:42 *** 03-03-2024---13:08,593,9.558179,7.0,0.771131,desktop/mobile,Dataframe 2
1,03-02-2024---20:14 *** 03-02-2024---20:42,480,7.931250,7.0,0.727273,mobile/desktop,Dataframe 2
23,03-02-2024---20:42 *** 03-03-2024---13:23,450,7.717778,6.0,0.710900,desktop/desktop,Dataframe 1
26,03-02-2024---20:53 *** 03-02-2024---22:10,356,14.632022,13.0,0.398210,mobile/desktop,Dataframe 1


In [71]:
# Visualizza righe prive di una ricerca
new_df = df_differenze.drop(df_differenze[df_differenze['coppia_ricerche'].str.contains("20:42")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo ricerche,prezzi_inferiori
26,03-02-2024---20:53 *** 03-02-2024---22:10,356,14.632022,13.0,0.398210,mobile/desktop,Dataframe 1
10,03-02-2024---20:31 *** 03-02-2024---20:53,353,14.968839,14.0,0.393534,desktop/mobile,Dataframe 2
14,03-02-2024---20:31 *** 03-02-2024---22:41,352,16.718750,14.0,0.392419,desktop/mobile,Dataframe 2
12,03-02-2024---20:31 *** 03-02-2024---21:42,352,16.315341,14.0,0.398190,desktop/mobile,Dataframe 2
35,03-02-2024---21:42 *** 03-02-2024---22:10,344,14.805233,13.0,0.390465,mobile/desktop,Dataframe 1
15,03-02-2024---20:31 *** 03-03-2024---13:08,338,16.582840,14.0,0.438961,desktop/mobile,Dataframe 2
32,03-02-2024---21:30 *** 03-02-2024---22:41,336,15.925595,14.0,0.387991,desktop/mobile,Dataframe 2
30,03-02-2024---21:30 *** 03-02-2024---21:42,330,15.627273,13.0,0.389151,desktop/mobile,Dataframe 2
40,03-02-2024---22:10 *** 03-03-2024---13:08,327,15.159021,14.0,0.424125,desktop/mobile,Dataframe 2
39,03-02-2024---22:10 *** 03-02-2024---22:41,326,15.236196,14.0,0.364653,desktop/mobile,Dataframe 2


In [72]:
mobile = 0
desktop = 0
for i in df_list:
    orario1 = i
    df1 = df[df["TIMESTAMP"]==orario1]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print(mobile,desktop)

['mobile']
['desktop']
['desktop']
['mobile']
['desktop']
['mobile']
['desktop']
['mobile']
['mobile']
['desktop']
5 5
